# Проблема с печеньками

Этот блокнот является частью [Bite Size Bayes](https://allendowney.github.io/BiteSizeBayes/), введения в вероятность и байесовскую статистику с использованием Python.

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>

<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/быстрое%20введение%20в%20pandas/Законы%20вероятности.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

Следующая ячейка загружает файл `utils.py`, содержащий некоторую полезную функцию, которая нам понадобится:

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/BiteSizeBayes/raw/master/utils.py')

Если все, что нам нужно, установлено, следующая ячейка должна работать без ошибок:

In [2]:
import pandas as pd
import numpy as np

from utils import values

## Обзор

В предыдущем блокноте я представил и доказал (вроде как) три теоремы вероятности:

**Теорема 1** дает нам новый способ вычисления условной вероятности с помощью конъюнкции:

$P(A|B) = \frac{P(A~\mathrm{and}~B)}{P(B)}$ 

**Теорема 2** дает нам новый способ вычисления конъюнкции с использованием условной вероятности:

$P(A~\mathrm{and}~B) = P(B) P(A|B)$

**Теорема 3**, также известная как теорема Байеса, дает нам способ перейти от $P(A|B)$ к $P(B|A)$ или наоборот:

$P(A|B) = \frac{P(A) P(B|A)}{P(B)}$


В примерах, которые мы видели до сих пор, эти теоремы нам действительно не нужны, потому что, когда у вас есть все данные, вы можете вычислить любую вероятность, какую хотите, любую конъюнкцию или любую условную вероятность, простым подсчетом.

Начиная с этого блокнота, мы рассмотрим примеры, в которых у нас нет всех данных, и увидим, что эти теоремы полезны, особенно теорема 3.

## Теорема Байеса

Есть два способа думать о теореме Байеса:

* Это стратегия "разделяй и властвуй" для вычисления условных вероятностей. Если сложно вычислить $P(A|B)$ напрямую, иногда проще вычислить условия с другой стороны уравнения: $P(A)$, $P(B|A)$ и $P(B)$.

* Это также способ обновления убеждений в свете новых данных.

Когда мы работаем со второй интерпретацией, мы часто записываем теорему Байеса с разными переменными. Вместо $A$ и $B$ мы используем $H$ и $D$, где

* $H$ означает "гипотеза", а

* $D$ означает "данные".

Итак, запишем теорему Байеса:

$P(H|D) = P(H) ~ P(D|H) ~/~ P(D)$

В этом контексте у каждого термина есть имя:

* $P(H)$ - это *"априорная вероятность"* гипотезы, которая показывает, насколько вы уверены, что $H$ истинно до просмотра данных,

* $P(D|H) $ - это *"правдоподобие" данных*, то есть вероятность увидеть $D$, если гипотеза верна,

* $P(D)$ - это *"полная вероятность данных"* (нормализует вероятность), то есть шанс увидеть $D$ независимо от того, является ли $H$ истинным или нет,

* $P(H|D)$ - это "апостериорная вероятность" гипотезы, которая показывает, насколько вы должны быть уверены в том, что $H$ истинно после учета данных.

Пример это прояснит.

## Проблема с печеньками

Вот проблема, которую я давным-давно узнал из Википедии, но теперь ее отредактировали.

> Предположим, у вас есть две миски с печеньем. Первая миска содержит 30 ванильных и 10 шоколадных печений. Во второй миске по 20 штук каждого вида.
>
> Вы наугад выбираете одну из мисок и, не глядя в миску, выбираете наугад одно из печений. Получается ванильное печенье.
>
> Каков шанс, что вы выбрали первую миску?

Предположим, что был равный шанс выбрать любую миску и равный шанс выбрать любое печенье в миске.

Мы можем решить эту проблему, используя теорему Байеса. 

Сначала я определю $H$ и $D$:

* $H$ - это гипотеза, что вы выбрали первую миску,

* $D$ - это исходная информация о том, что печенька является ванильной.

Нам нужна апостериорная вероятность $H$, которая равна $P(H|D)$. Не очевидно, как вычислить ее напрямую, но если мы сможем вычислить условия в правой части теоремы Байеса, то сможем добраться до нее косвенно.

1. $P(H)$ - это априорная вероятность $H$, которая представляет собой вероятность выбора первой миски до того, как мы увидим данные. Если есть равные шансы выбрать любую миску, $P(H)$ будет $1/2$.

2. $ P(D|H)$ - это правдоподобие данных, то есть вероятность получения ванильной печеньки, если значение $H$ истинно, другими словами, вероятность получения ванильной печеньки из первой миски, т.е. $30/40$ или $3/4$.

3. $P(D)$ - это полная вероятность данных, которая представляет собой шанс получить ванильную печеньку независимо от того, является ли $H$ истинной или нет. В этом примере мы можем вычислить $P(D)$ напрямую: поскольку миски одинаково вероятны и содержат одинаковое количество печений, вы с одинаковой вероятностью выберете любую печеньку. Объединяя две миски, получается 50 ванильных и 30 шоколадных печений, поэтому вероятность выбора ванильного печенья составляет $50/80$ или $5/8$.

Теперь, когда у нас есть условия в правой части, мы можем использовать теорему Байеса, чтобы объединить их:

In [3]:
prior = 1/2
prior

0.5

In [4]:
likelihood = 3/4
likelihood

0.75

In [5]:
prob_data = 5/8
prob_data

0.625

In [6]:
posterior = prior * likelihood / prob_data
posterior

0.6

Апостериорная вероятность составляет $0.6$, что немного выше, чем предыдущая, которая составляла $0.5$.

Таким образом, ванильное печенье дает нам больше уверенности в том, что мы выбрали первую миску.

**Упражнение:** Что, если бы вместо этого мы выбрали шоколадное печенье; какова будет апостериорная вероятность первой миски?

In [7]:
# Решение здесь

## Доказательство

В предыдущем примере и упражнении обратите внимание на закономерность:

* Ванильное печенье более вероятно, если мы выберем первую миску, поэтому получение ванильного печенья делает первую миску более вероятной.

* Шоколадное печенье будет менее вероятным, если мы выберем первую миску, поэтому получение шоколадного печенья сделает первую миску менее вероятной.

Если данные повышают вероятность гипотезы, мы говорим, что это "свидетельство в пользу" гипотезы.

Если данные снижают вероятность гипотезы, это "свидетельство против" гипотезы.

Приведем еще один пример:

> Предположим, у вас в коробке две монеты. Одна - обычная монета с орлами на одной стороне и решками с другой, а другая - хитрая с орлами с обеих сторон.
>
> Вы выбираете монету наугад и видите, что одна из сторон - орел. Являются ли эти данные свидетельством в пользу или против гипотезы о том, что вы выбрали хитрую монету?

Посмотрите, сможете ли вы найти ответ, прежде чем читать мое решение. Предлагаю следующие шаги:

1. Во-первых, четко сформулируйте гипотезу и данные.

2. Затем подумайте об априорности, правдоподобии и общей вероятности данных.

3. Примените теорему Байеса, чтобы вычислить апостериорную вероятность гипотезы.

4. Используйте результат, чтобы ответить на поставленный вопрос.

В этом примере:

* $H$ - это гипотеза о том, что вы выбрали хитрую монету с двумя орлами.

* $D$ - это наблюдение, что одна сторона медали - орел.

Теперь давайте подумаем о правосторонних условиях:

* Априорная вероятность - 1/2, потому что мы с равной вероятностью выберем любую монету.

* Правдоподобие данных равно 1, потому что, если мы выберем хитрую монету, то обязательно увидим орла.

* Полная вероятность данных составляет 3/4, потому что 3 из 4 сторон являются орлами, и мы с равной вероятностью увидим любую из них.

Вот что мы получим, если применим теорему Байеса:

In [8]:
prior = 1/2
likelihood = 1
prob_data = 3/4

posterior = prior * likelihood / prob_data
posterior

0.6666666666666666

Апостериорная величина больше, чем априорная, поэтому эти данные свидетельствуют в пользу гипотезы о том, что вы выбрали хитрую монету.

И в этом есть смысл, потому что вероятность выпадения орла выше, если вы выберете хитрую, а не обычную монету.

## Таблица Байеса

В проблеме печений и монет мы могли вычислить вероятность данных напрямую, но это не всегда так. Фактически, вычисление полной вероятности данных часто является самой сложной частью проблемы.

К счастью, есть еще один способ решения подобных проблем, который упрощает задачу: *таблица Байеса*.

Вы можете написать таблицу Байеса на бумаге или использовать электронную таблицу, но в этом блокноте я буду использовать фреймы данных библиотки pandas.

Сначала я займусь проблемой печений. 

Вот пустой фрейм данных с одной строкой для каждой гипотезы:

In [10]:
import pandas as pd

table = pd.DataFrame(index=['Bowl 1', 'Bowl 2'])

Теперь я добавлю столбец для представления априорных значений:

In [11]:
table['prior'] = 1/2, 1/2
table

,prior
Bowl 1,0.5
Bowl 2,0.5


И столбец для правдоподобия:

In [12]:
table['likelihood'] = 3/4, 1/2
table

,prior,likelihood
Bowl 1,0.5,0.75
Bowl 2,0.5,0.50


Здесь мы видим отличие от предыдущего метода: мы вычисляем правдоподобие для обеих гипотез, а не только для первой миски:

* Вероятность получить ванильное печенье из первой миски составляет 3/4.

* Шанс получить ванильное печенье из второй миски - 1/2.

Следующий шаг аналогичен тому, что мы сделали с теоремой Байеса; мы умножаем априорные значения на правдоподобие:

In [13]:
table['unnorm'] = table['prior'] * table['likelihood']
table

,prior,likelihood,unnorm
Bowl 1,0.5,0.75,0.375
Bowl 2,0.5,0.50,0.250


Я назвал результат `unnorm`, потому что он "ненормализованный апостериорный" (unnormalized posterior).

Чтобы понять, что это означает, давайте сравним правую часть теоремы Байеса:

$P(H) P(D|H)~/~P(D)$

К тому, что мы вычислили до сих пор:

$P(H) P(D|H)$

Разница в том, что мы не разделили на $P(D)$ полную вероятность данных. Так что давай сделаем это.

Есть два способа вычислить $P(D)$:

1. иногда мы можем выяснить ее напрямую;

2. в противном случае мы можем вычислить ее, сложив ненормализованные апостериоры (`unnorm`).

С помощью вычислений я покажу второй способ, а затем объясню, как он работает.

Вот общее количество `unnorm`:

In [15]:
prob_data = table['unnorm'].sum()
prob_data

0.625

Обратите внимание, что мы получаем 5/8, что мы и получили, напрямую вычислив $P(D)$.

Теперь разделим на $P(D)$, чтобы получить апостериорную вероятность:

In [16]:
table['posterior'] = table['unnorm'] / prob_data
table

,prior,likelihood,unnorm,posterior
Bowl 1,0.5,0.75,0.375,0.6
Bowl 2,0.5,0.50,0.250,0.4


Апостериорная вероятность для первой миски равна 0,6, что мы и получили, явно используя теорему Байеса.

В качестве бонуса мы также получаем апостериорную вероятность второй миски, равную 0,4.

Сумма апостериорных вероятностей дает 1, что должно быть, потому что гипотезы "дополняют друг друга"; то есть либо одно из них истинно, либо другое, но не оба. Таким образом, их вероятности должны составлять в сумме 1.

Когда мы складываем ненормализованные апостериорные элементы и делим их, мы заставляем дополнять апостериорные элементы до 1. Этот процесс называется "нормализацией", поэтому полная вероятность данных также называется ["нормализующей константой"](https://en.wikipedia.org/wiki/Normalizing_constant#Bayes'_theorem).

Возможно, еще не ясно, почему ненормализованные апостериорные элементы в сумме составляют $P(D)$. Я вернусь к этому в следующем блокноте.

**Упражнение:** Решите проблему с монеткой, используя таблицу Байеса:

> Допустим, у вас в коробке две монеты. Одна - обычная монета с орлами на одной стороне и решками с другой, а другая - хитрая с орлами с обеих сторон.
>
> Вы выбираете монету наугад и видите, что одна из сторон - орел. Какова апостериорная вероятность того, что вы выбрали хитрую монету?

*Подсказка*: ответ все равно должен быть 2/3.

In [17]:
# Решение здесь

## Итоги



В этом блокноте я представил две проблемы: проблему с печеньками и проблему с монеткой.

Мы решили обе проблемы, используя теорему Байеса; затем я представил таблицу Байеса - метод решения проблем, в которых трудно вычислить полную вероятность данных напрямую.

В следующем блокноте мы увидим примеры с более чем двумя гипотезами, и я объясню более внимательно, как работает таблица Байеса.

<a href="https://t.me/init_python"><img src="https://dfedorov.spb.ru/pandas/logo-telegram.png" width="35" height="35" alt="telegram" align="left"></a>